In [ ]:
# if the libraries are not installed
!pip install tflearn
!pip install tensorflow

In [1]:
# things we need for NLP
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

#import the intents file
import json
with open('intents.json') as intents_file:
    intents = json.load(intents_file)

# for saving data
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\himas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\himas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


# Organize the words in each pattern array of the intents dictionary into:
            doc: a document array containg the processed words along with their tag
            words: an array of all the processed words present in the document
            classes: an array of all tags present in the document

In [2]:
words = []
classes = []
doc = []
# create a list of words that should be ignored as they do not add value to the model
ignore_words = ['?','!',',',"'s",".","where","who","is","the","this","in","you","so","for","me","how","want",
                "your", "a","that","there","to","of","i","and","what","with","up","do","did","be","can","could"]

#loop through each pattern in the intents dictionary
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word in the sentence
        token_w = nltk.word_tokenize(pattern)
        #lemmatize and lower each word if it is not one among the ignore_words array
        w = [lemmatizer.lemmatize(word.lower()) for word in token_w if lemmatizer.lemmatize(word.lower()) not in ignore_words]
        # add to the words list
        words.extend(w)
        #add to doc in the corpus
        doc.append((w, intent['tag']))
        #add the tag to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# remove duplicates in the words list
words = sorted(list(set(words)))

print (len(doc),'documents')
print (len(classes),'classes', classes)
print (len(words), 'unique stemmed words', words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


90 documents
14 classes ['greeting', 'goodbye', 'thanks', 'STEMforall', 'history', 'partners', 'contact', 'donations', 'volunteer', 'overview_of_positions', 'lead', 'assistant', 'tutor_application', 'enrollment']
76 unique stemmed words ['about', 'achieve', 'achievement', 'an', 'anyone', 'application', 'apply', 'applying', 'are', 'assistant', 'available', 'awesome', 'bye', 'chatting', 'class', 'contact', 'course', 'day', 'donate', 'donating', 'donation', 'enroll', 'enrolling', 'far', 'get', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'info', 'information', 'instructor', 'interested', 'later', 'lead', 'main', 'more', 'next', 'nice', 'offered', 'option', 'organization', 'overview', 'partner', 'position', 'provide', 'requirement', 'see', 'service', 'sign', 'sign-up', 'signing', 'sponsor', 'sponsored', 'stemforall', 'support', 'ta', 'teach', 'teacher', 'teaching', 'tell', 'thank', 'thanks', 'till', 'time', 'tutor', 'volunteer', 'volunte

# Transform from doc of words to tensors of numbers

Once we have our training data, the next step is to transform it into a format that is recognized by the model. In this case, the data in the doc is transformed such that:
    
    input: bag_of_words - a bag of words tensor of zeros and ones, 
    
          where one indicates that the corresponding word in the words array is present in this pattern and zero, otherwise.
    
    output: output_vec - an output vector tensor of zeros and ones, 
    
          where one indicates that the corresponding class in the classes array is the output for the given bag of words.

In [3]:
# create empty arrays for storing data
train_data = []
output = []
# Initialize an empty vector for the output
empty_vec = [0]*len(classes)

# fill training data by obtaining bag of words for each sentence in doc
for sent in doc:
    #initialize the bag of words array
    bag_of_words = []
    # get the list of lemmatized words 
    lemmatized_words = sent[0]
    # create bag of words array
    for word in words:
        bag_of_words.append(1) if word in lemmatized_words else bag_of_words.append(0)

    # create output vector such that it has '1' for current tag and '0' for the remaining tags
    output_vec = list(empty_vec)
    output_vec[classes.index(sent[1])] = 1

    train_data.append([bag_of_words, output_vec])

# shuffle the features and transform it to an np.array
random.shuffle(train_data)
train_data = np.array(train_data)

#create the X (patterns), y (intents) lists for the model
train_X = list(train_data[:,0])
train_y = list(train_data[:,1])

print('training data created\n')
example_idx = random.randint(0,len(doc))
print('train_X example:', train_X[example_idx])
print('train_y example:', train_y[example_idx])

training data created

train_X example: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
train_y example: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Building the neural network and training the model

In [ ]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()

# build the neural network
net = tflearn.input_data(shape=[None, len(train_X[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#train the model (applying gradient descent algorithm)
model.fit(train_X, train_y, n_epoch=1000, batch_size=8, show_metric=True)

model.save('model.tflearn')